In [1]:
import random
import re
import collections
import math
from scipy.stats import multivariate_normal
import numpy as np

In [2]:
def load_observations():

    pattern = re.compile(r'''
    \s*(?P<object_id>\d+),            # object_id (first number)
    \s*(?P<within_frame_id>\d+),       # within_frame_id (second number)
    \s*'(?P<file_path>[^']+)',              # file path (file location)
    \s*cX\s*=\s*(?P<x>\d+),                 # cX coordinate
    \s*cY\s*=\s*(?P<y>\d+),                 # cY coordinate
    \s*Frame\s*=\s*(?P<frame>\d+)           # Frame number
''', re.VERBOSE)
    
    observations = collections.defaultdict(list)    
    with open('ObjectXYs.txt') as object_xys:
        for line in object_xys:
            m = pattern.match(line)
            if m:
                obj_id = int(m.group('object_id'))
                frame = m.group('frame')
                cX=m.group('x')
                cY=m.group('y')
                
                observations[int(m.group('object_id'))].append((int(m.group('frame')), int(m.group('x')), int(m.group('y'))))
                
    return observations

In [3]:
obs = load_observations()
print(obs)

defaultdict(<class 'list'>, {1: [(150, 1028, 980), (151, 1158, 980), (152, 1287, 979), (153, 1415, 979), (154, 1542, 979), (155, 1668, 978), (156, 1793, 979), (157, 1916, 978), (158, 2040, 978), (159, 2162, 977), (160, 2284, 977), (161, 2405, 976), (162, 2526, 976), (163, 2645, 976), (164, 2763, 975), (165, 2881, 974), (166, 2998, 974), (167, 3114, 973), (168, 3230, 973), (169, 3343, 971), (170, 3455, 968), (171, 3567, 964), (172, 3680, 954), (173, 3800, 944), (174, 3921, 936), (175, 4046, 931)], 2: [(150, 1550, 1602), (151, 1682, 1603), (152, 1813, 1604), (153, 1945, 1605), (154, 2076, 1606), (155, 2206, 1607), (156, 2336, 1607), (157, 2465, 1610), (158, 2593, 1610), (159, 2720, 1611), (160, 2848, 1611), (161, 2975, 1613), (162, 3103, 1615), (163, 3231, 1616), (164, 3357, 1620), (165, 3485, 1624), (166, 3613, 1628), (167, 3743, 1631), (168, 3877, 1634), (169, 4010, 1635)], 3: [(158, 222, 933), (159, 325, 932), (160, 424, 932), (161, 523, 931), (162, 621, 931), (163, 718, 931), (164, 8

In [4]:
def filtered_observations(obs,obj_id):
    return {obj_id: obs.get(obj_id, [])}

In [46]:
pseudo_obs=filtered_observations(obs,5)
print(len(pseudo_obs))
print(pseudo_obs)

1
{5: [(175, 290, 507), (176, 398, 505), (177, 505, 504), (178, 614, 505), (179, 721, 505), (180, 829, 506), (181, 935, 507), (182, 1038, 507), (183, 1138, 505), (184, 1240, 504), (185, 1343, 505), (186, 1445, 506), (187, 1546, 507), (188, 1648, 507), (189, 1752, 507), (190, 1856, 505), (191, 1959, 504), (192, 2062, 503), (193, 2165, 504), (194, 2267, 504), (195, 2368, 505), (196, 2466, 503), (197, 2563, 502), (198, 2659, 500), (199, 2756, 498), (200, 2852, 497), (201, 2949, 497), (202, 3047, 496), (203, 3146, 495), (204, 3245, 493), (205, 3345, 491), (206, 3444, 489), (207, 3541, 487), (208, 3643, 486), (209, 3745, 486), (210, 3846, 487), (211, 3945, 490), (212, 4043, 494), (213, 4126, 509)]}


In [54]:
def grid_displacement_observation(pseudo_obs): 
    MAX_X=2000
    MAX_Y=4000
    
    GRID_SQUARES = 2

    grid_obs = [[[] for _ in range(GRID_SQUARES)] for _ in range(GRID_SQUARES)]
    grid_stats = [[(0, 0, 0) for _ in range(GRID_SQUARES)] for _ in range(GRID_SQUARES)]
    #print(grid_obs)

    for obj_id, obs in pseudo_obs.items():
        grid_cells_visited = set()
        for j in range(len(obs)-1):
            x1_row=math.floor(obs[j][1]/4000*GRID_SQUARES)
            y1_col=math.floor(obs[j][2]/2000*GRID_SQUARES)

            x2_row=math.floor(obs[j+1][1]/4000*GRID_SQUARES)
            y2_col=math.floor(obs[j+1][2]/2000*GRID_SQUARES)

            #print(f"contributing row is: {x1_row}, col is: {y1_col}")
            #print(f"next row is: {x2_row}, col is: {y2_col}")
            if(0<= x1_row <2 and 0<= y1_col <2):
                grid_pos=grid_obs[x1_row][y1_col]
           

                if(obs[j+1][0]==obs[j][0]+1):
                    dx=obs[j+1][1]-obs[j][1]
                    dy=obs[j+1][2]-obs[j][2]
                    grid_pos.append((dx,dy))
                    sum_x, sum_y, count = grid_stats[x1_row][y1_col]
                    grid_stats[x1_row][y1_col] = (sum_x + dx, sum_y + dy, count + 1)
               
                #print(f"displacement is: {dx,dy}")
                    grid_cells_visited.add((x1_row,y1_col))
                else:
                    frame_distance=(obs[j][0]+1)-obs[j+1][0] 
                    if (frame_distance!=0):
                        dx=(obs[j+1][1]-obs[j][1])/frame_distance
                        dy=(obs[j+1][2]-obs[j][2])/frame_distance
                        grid_pos.append((dx,dy))
                        grid_cells_visited.add((x1_row,y1_col))
                    
                        sum_x, sum_y, count = grid_stats[x1_row][y1_col]
                        grid_stats[x1_row][y1_col] = (sum_x + dx, sum_y + dy, count + 1)
                    #print(f"displacement for skipping frame is: {dx,dy}")

            #print(f"for object {obj_id} observations no {j} observation is {obs[j][0],obs[j][1],obs[j][2]}")
            #print(f"for object {obj_id} observations no {j+1} observation is {obs[j+1][0],obs[j+1][1],obs[j+1][2]}")
    #print(grid_stats)
    #print(grid_cells_visited)
    return grid_obs,grid_stats

In [55]:
grid_displacements,grid_sums=grid_displacement_observation(pseudo_obs)
print(grid_displacements)

[[[(108, -2), (107, -1), (109, 1), (107, 0), (108, 1), (106, 1), (103, 0), (100, -2), (102, -1), (103, 1), (102, 1), (101, 1), (102, 0), (104, 0), (104, -2), (103, -1), (103, -1)], []], [[(103, 1), (102, 0), (101, 1), (98, -2), (97, -1), (96, -2), (97, -2), (96, -1), (97, 0), (98, -1), (99, -1), (99, -2), (100, -2), (99, -2), (97, -2), (102, -1), (102, 0), (101, 1), (99, 3), (98, 4)], []]]


In [59]:
def grid_covariance_calculate(grid_displacements):
    covar=[]
    GRID_SQUARES = 2
    grid_stats = [[None for _ in range(GRID_SQUARES)] for _ in range(GRID_SQUARES)]
    
    for i in range(GRID_SQUARES):
        for j in range(GRID_SQUARES):
            pdfs_for_cells=[]
            if(len(grid_displacements[i][j])>1):
                dxdy_items = np.array(grid_displacements[i][j])
                #print(f" for {i,j} cell displacements are {dxdy_items}, {dxdy_items.shape}")

                mu = np.mean(dxdy_items, axis=0)
                #print(mu,mu.shape)
                
                cov_matrix = np.cov(dxdy_items.T)
                #print(cov_matrix,cov_matrix.shape,cov_matrix.T,cov_matrix.T.shape)
                    
                for point_to_evaluate in dxdy_items:
                    pdf_value = multivariate_normal.pdf(point_to_evaluate, mean=mu, cov=cov_matrix, allow_singular=True)
                    pdfs_for_cells.append(pdf_value)
                    #print(f"current point: {point_to_evaluate}, {point_to_evaluate.shape}")
                print(pdfs_for_cells)
              
            else:
                mu = (0, 0)
                cov_matrix = np.zeros((2, 2))
                pdf_values = 0
                pdfs_for_cells.append(pdf_values)
                
            log_pdf_sum=sum(math.log(pdf) for pdf in pdfs_for_cells if pdf!=0)
            print(f"log sum of the pdfs: {log_pdf_sum}")
            grid_stats[i][j] = {
                'mu': mu,
                'cov_matrix': cov_matrix,
                'pdf_values': pdfs_for_cells,
                'log_sum': log_pdf_sum
        }
            
    #print(grid_stats)
    return grid_stats                



In [60]:
grid_pdfs=grid_covariance_calculate(grid_displacements)
print(grid_pdfs)

log sum of the pdfs: 0
log sum of the pdfs: 0
log sum of the pdfs: 0
log sum of the pdfs: 0
[[{'mu': (0, 0), 'cov_matrix': array([[0., 0.],
       [0., 0.]]), 'pdf_values': [0], 'log_sum': 0}, {'mu': (0, 0), 'cov_matrix': array([[0., 0.],
       [0., 0.]]), 'pdf_values': [0], 'log_sum': 0}], [{'mu': (0, 0), 'cov_matrix': array([[0., 0.],
       [0., 0.]]), 'pdf_values': [0], 'log_sum': 0}, {'mu': (0, 0), 'cov_matrix': array([[0., 0.],
       [0., 0.]]), 'pdf_values': [0], 'log_sum': 0}]]


In [61]:
for i in range (100):
    pseudo_obs=filtered_observations(obs,i+1)
    print(f"for object id: {i+1}")
    print(pseudo_obs)
    grid_displacements,grid_sums=grid_displacement_observation(pseudo_obs)
    print(grid_displacements)
    grid_pdfs=grid_covariance_calculate(grid_displacements)
    print(grid_pdfs)

for object id: 1
{1: [(150, 1028, 980), (151, 1158, 980), (152, 1287, 979), (153, 1415, 979), (154, 1542, 979), (155, 1668, 978), (156, 1793, 979), (157, 1916, 978), (158, 2040, 978), (159, 2162, 977), (160, 2284, 977), (161, 2405, 976), (162, 2526, 976), (163, 2645, 976), (164, 2763, 975), (165, 2881, 974), (166, 2998, 974), (167, 3114, 973), (168, 3230, 973), (169, 3343, 971), (170, 3455, 968), (171, 3567, 964), (172, 3680, 954), (173, 3800, 944), (174, 3921, 936), (175, 4046, 931)]}
[[[(130, 0), (129, -1), (128, 0), (127, 0), (126, -1), (125, 1), (123, -1), (124, 0)], []], [[(122, -1), (122, 0), (121, -1), (121, 0), (119, 0), (118, -1), (118, -1), (117, 0), (116, -1), (116, 0), (113, -2), (112, -3), (112, -4), (113, -10), (120, -10), (121, -8), (125, -5)], []]]
[0.031100961493761268, 0.031100961493761254, 0.07156256258073075, 0.0845411784760957, 0.05127681675398981, 0.015967766049270485, 0.018863686692508222, 0.05127681675398981]
log sum of the pdfs: -26.0974656859472
log sum of the